In [1]:
import pandas as pd
from sklearn.neighbors import LocalOutlierFactor
from numpy import quantile
import warnings

warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)

In [2]:
df = pd.read_csv(r'..\..\datasets\df_rob_norm.csv', encoding='latin-1')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2065547 entries, 0 to 2065546
Data columns (total 7 columns):
 #   Column            Dtype  
---  ------            -----  
 0   nf_timestamp      float64
 1   emit_lat          float64
 2   emit_long         float64
 3   prod_ncm          float64
 4   prod_quant        float64
 5   prod_valor_unit   float64
 6   prod_valor_total  float64
dtypes: float64(7)
memory usage: 110.3 MB


In [3]:
model = LocalOutlierFactor(n_neighbors=20) 
model.fit_predict(df)
df['anomaly_score'] = model.negative_outlier_factor_ 

outlier_thresh = 0.0001

thresh = quantile(df['anomaly_score'], outlier_thresh)

print(thresh)

df_outliers = df[df['anomaly_score'] <= thresh]

df_outliers

-1723623218.8060634


,nf_timestamp,emit_lat,emit_long,prod_ncm,prod_quant,prod_valor_unit,prod_valor_total,anomaly_score
262933,-0.502401,-0.307792,0.000000,1.024078,2.916667,-0.144330,0.915592,-1.840572e+09
321666,-0.378757,-0.307792,0.000000,1.295271,2.916667,-0.117182,1.270159,-1.808239e+09
343544,-0.388275,-0.307792,0.000000,1.317258,2.916667,-0.154639,0.780946,-1.960193e+09
474195,-0.380328,-0.307792,0.000000,1.069557,3.291667,-0.140893,1.116663,-1.956768e+09
533761,-0.421360,-0.307792,0.000000,1.317446,2.916667,-0.120275,1.229766,-1.827935e+09
...,...,...,...,...,...,...,...,...
2064098,0.582504,-42.166020,-3.578734,-0.042219,20.625000,0.261168,43.344005,-4.317711e+10
2064824,-0.225657,-2.647967,-0.071263,1.316231,-0.187500,-0.137801,-0.332156,-2.896295e+09
2065137,0.827172,-41.151632,-5.254226,1.296087,-0.187500,236.872852,20.304599,-4.993170e+10
2065380,-1.007208,-67.721084,-7.817063,1.091633,-0.187500,37.903780,2.980162,-2.602441e+10


In [4]:
df_outliers['idx'] = df_outliers.index
df_outliers = df_outliers.sort_values(by='anomaly_score', ascending=True)
df_outliers.to_csv('output\outliers.csv', index=False)
df_outliers

,nf_timestamp,emit_lat,emit_long,prod_ncm,prod_quant,prod_valor_unit,prod_valor_total,anomaly_score,idx
1942852,0.695183,-49.241908,-4.277001,-0.063497,-0.145833,1543.699656,402.956165,-6.739464e+11,1942852
1942893,0.115000,-49.241908,-4.277001,-0.063497,-0.145833,1543.699656,402.956165,-6.739445e+11,1942893
1625205,-0.083046,-49.058145,-4.219851,1.128106,-0.187500,1579.356701,137.196014,-2.047600e+11,1625205
2065437,0.875439,-67.799010,-5.743649,1.070370,-0.187500,1566.769759,136.100057,-1.960747e+11,2065437
2055257,-0.060357,-45.747258,-3.351696,1.128110,-0.187500,1576.754639,136.969450,-1.919499e+11,2055257
...,...,...,...,...,...,...,...,...,...
1054912,0.817560,-11.523536,-1.470334,1.129253,-0.187500,4.872852,0.104126,-1.785272e+09,1054912
1077051,-0.423791,-2.797626,0.389299,1.067824,-0.187500,11.786942,0.706143,-1.782076e+09,1077051
1087231,-0.540012,-2.797626,0.389299,0.836319,-0.187500,12.817869,0.795907,-1.777948e+09,1087231
540577,-0.630969,-0.307792,0.000000,1.193980,2.916667,-0.140893,0.960474,-1.743865e+09,540577
